<a href="https://colab.research.google.com/github/Jihyozhixiao/YBIGTA/blob/master/%EA%B9%80%EC%A7%80%ED%9A%A8_Hyperparameter_Tuning_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning 과제

- Manual Search
- Random Search (Grid Search 참고)
- Bayesian Optimization using HyperOpt
<br>연습해보기!

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import load_iris
import warnings
warnings.filterwarnings('ignore')
import time
from functools import partial

In [ ]:
iris = load_iris()

X = iris.data
y = iris.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

## 1. Manual Search
사이킷런 Logistic Regresstion https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logistic%20regression#sklearn.linear_model.LogisticRegression

In [ ]:
'''
maunal 1 
'''

manual1 = LogisticRegression(
    penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None
)

# 다 default 값일 때 값
manual1.fit(X_train,y_train)
y_pred = manual1.predict(X_train)

# 정확도를 계산해주세요
accuracy = accuracy_score(y_train, y_pred)
accuracy

0.9642857142857143

In [ ]:
'''
maunal 2 
'''
# 다양한 하이퍼파라미터 값을 사용하여 또 다른 manual search를 해보세요
# manual1 에 비해 정확도가 더 높아지면 더 좋습니다 ! 낮아졌어요,,,

manual2 = LogisticRegression(
    penalty='elasticnet', dual=False, tol=0.001, C=2.0, fit_intercept=False, intercept_scaling=1, class_weight=None, random_state=None, solver='saga', max_iter=100, multi_class='auto', verbose=0, warm_start=True, n_jobs=None, l1_ratio=0
)

manual2.fit(X_train,y_train)
y_pred = manual2.predict(X_train)

# 정확도를 계산해주세요
accuracy = accuracy_score(y_train, y_pred)
accuracy

0.9553571428571429

## 2. Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 그리드 서치 호출
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
grid_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {grid_search.score(X_test, y_test)}")

print(f"best parameters: {grid_search.best_params_}")
print(f"best CV score: {grid_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)

test set score : 1.0
best parameters: {'C': 1, 'max_iter': 100}
best CV score: 0.9553359683794467
time : 7.331149339675903


## 3. Random Search
grid search 코드와 매우 유사하니 참고해서 빈칸 채워주세요 !

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 파라미터 후보 설정
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'max_iter' : [100, 500, 1000, 1500, 2000]}

# 랜덤 서치 호출
random_search = RandomizedSearchCV(LogisticRegression(), param_grid, cv=5, return_train_score=True)   

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

start = time.time()
# 모델 적합
random_search.fit(X_train, y_train)

# 모델 평가
print(f"test set score : {random_search.score(X_test, y_test)}")

print(f"best parameters: {random_search.best_params_}")
print(f"best CV score: {random_search.best_score_}")

# 소요 시간 측정
print("time :", time.time() - start)


test set score : 0.9736842105263158
best parameters: {'max_iter': 100, 'C': 10}
best CV score: 0.9553359683794467
time : 2.529472827911377


## 4. HyperOpt

In [ ]:
!pip install hyperopt

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

space = {
    'penalty' : hp.choice('penalty', ['l2', 'none']), 
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'max_iter' : hp.uniform('max_iter', 100,1000)
}

def objective(space):
    hopt = LogisticRegression(penalty = space['penalty'], 
                                fit_intercept = space['fit_intercept'],
                                 max_iter = space['max_iter']
                                 )
    accuracy = cross_val_score(hopt, X_train, y_train, cv=4).mean()
    return {'loss': -accuracy, 'status': STATUS_OK }
    
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 20,
            trials= trials)
best

100%|██████████| 20/20 [00:02<00:00,  8.92it/s, best loss: -0.9553571428571429]


{'fit_intercept': 0, 'max_iter': 421.1721225436772, 'penalty': 0}

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

pen = {0: 'l1', 1: 'l2', 2: 'elasticnet', 3: None}
incpt = {0: True, 1: False}

trained = LogisticRegression(penalty = pen[best['penalty']],
                             fit_intercept = incpt[best['fit_intercept']],
                             solver='liblinear'
                            ).fit(X_train,y_train)


hopt_pf = trained.predict(X_test)
hopt_acc = accuracy_score(y_test,hopt_pf)
print(confusion_matrix(y_test,hopt_pf))
print(classification_report(y_test,hopt_pf))
print(hopt_acc)

[[16  0  0]
 [ 0 11  0]
 [ 0  1 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.92      1.00      0.96        11
           2       1.00      0.91      0.95        11

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38

0.9736842105263158


## 선택) Bayesian Optimization 사용


In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=1a9aa838d770474b3d1b1a124eeb0910048eb1b0269e6bb6844a29b7c1364fb7
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import MinMaxScaler

pbounds = {'max_depth': (3, 7),
                'learning_rate': (0.01, 0.2),
                'n_estimators': (5000, 10000),
                'gamma': (0, 100),
                'min_child_weight': (0, 3),
                'subsample': (0.5, 1),
                'colsample_bytree' :(0.2, 1)}

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def black_box_function(C):
    # C: SVC hyper parameter to optimize for.
    model = SVC(C = C)
    model.fit(X_train_scaled, y_train)
    y_score = model.decision_function(X_test_scaled)
    f = roc_auc_score(y_test, y_score)
    return f

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1
)

optimizer.maximize(init_points=2, n_iter=30)
print(optimizer.max)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------


TypeError: ignored